In [14]:
%%capture
!pip install -r requirements.txt

In [4]:
from mlem.api import import_object, save
import torch
from torch import nn
import pandas as pd
import numpy as np
import torch

pd.set_option('display.max_columns', None)

# Build model with non-linear activation function
class InterruptionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=29, out_features=200)
        self.layer_2 = nn.Linear(in_features=200, out_features=100)
        self.layer_3 = nn.Linear(in_features=100, out_features=1)
        self.relu = nn.ReLU() # <- add in ReLU activation function
        # Can also put sigmoid in the model
        # This would mean you don't need to use it on the predictions
        # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Intersperse the ReLU activation function between layers
        return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))
        
path_to_pt = "../model-archiver/model-store/youtubegoes5g/model.pt"

model = InterruptionModel()
model.load_state_dict(torch.load(path_to_pt, weights_only=True))
model.eval()

# Step 1: Read the CSV file and drop the specified columns
sample = pd.read_csv('../data/external/dataset.csv', nrows=1).drop(columns=['Stall', 'ID', 'Quality', 'Time'], errors='ignore')

# Step 2: Replace ' ', '-', and np.nan with 0
sample = sample.replace([' ', '-', np.nan], 0)

# Convert all columns to float 
sample = sample.astype(float)

# Step 3: Extract the first sample as a NumPy array without the column names
first_sample = sample.values#.flatten()  # Use flatten() to get a 1D array

# Step 4: Convert the first sample to a PyTorch tensor
first_sample_tensor = torch.tensor(first_sample, dtype=torch.float32)

# Display the tensor
print("First Sample Tensor:", first_sample_tensor)

save(model, "models/youtubegoes5g", sample_data=first_sample_tensor)

#model = import_object(path="../model-archiver/model-store/youtubegoes5g/model.pt", target="models/youtubegoes5g.mlem", type_="pickle")



First Sample Tensor: tensor([[ 13.0000,  13.0000,  13.0000,   0.0000,  13.0000,  13.0000,  13.0000,
          13.0000, -76.0000, -76.0000, -81.0000, -76.0000, -78.5000, -76.0000,
         -76.0000,  -7.0000,  -7.0000, -12.0000,  -7.0000,  -9.5000,  -7.0000,
          -7.0000,  12.0000,  12.0000,   7.0000,  12.0000,   9.5000,  12.0000,
          12.0000]])


MlemModel(location=Location(path='/home/jovyan/mlops-workflow/local/models/youtubegoes5g.mlem', project=None, rev=None, uri='file:///home/jovyan/mlops-workflow/local/models/youtubegoes5g.mlem', project_uri=None, fs=<fsspec.implementations.local.LocalFileSystem object at 0x7f22afe91590>), params={}, artifacts={'data': LocalArtifact(uri='youtubegoes5g', size=110792, hash='74ba0722cab958b5bc4f9661ce0c851d')}, requirements=Requirements(__root__=[InstallableRequirement(module='torch', version='2.3.0', package_name=None, extra_index='https://download.pytorch.org/whl/cpu', source_url=None, vcs=None, vcs_commit=None)]), processors_cache={'model': TorchModel(model=InterruptionModel(
  (layer_1): Linear(in_features=29, out_features=200, bias=True)
  (layer_2): Linear(in_features=200, out_features=100, bias=True)
  (layer_3): Linear(in_features=100, out_features=1, bias=True)
  (relu): ReLU()
), io=TorchModelIO(is_jit=False), methods={'__call__': Signature(name='__call__', args=[], returns=TorchT

In [16]:
from mlem.api import load

model = load("models/youtubegoes5g")  # RandomForestClassifier

# Step 1: Read the first row of the CSV file
sample = pd.read_csv('../data/external/dataset.csv', nrows=1)

# Capture the value of the 'Stall' column before dropping it
stall_value = sample['Stall'].values[0] if 'Stall' in sample.columns else None
print("Stall Value:", stall_value)

# Step 2: Drop the 'Stall', 'ID', 'Quality', and 'Time' columns
sample = sample.drop(columns=['Stall', 'ID', 'Quality', 'Time'], errors='ignore')

# Step 3: Replace ' ', '-', and np.nan with 0
sample = sample.replace([' ', '-', np.nan], 0)

# Convert all columns to float
sample = sample.astype(float)

# Step 4: Extract the first sample as a NumPy array without the column names
first_sample = sample.values.flatten()  # Use flatten() to get a 1D array

# Step 5: Convert the first sample to a PyTorch tensor
first_sample_tensor = torch.tensor(first_sample, dtype=torch.float32)

# Display the tensor
print("First Sample Tensor:", first_sample_tensor)

#y_pred = model.forward(first_sample_tensor)

#y_logits = model(first_sample_tensor).squeeze()

#y_pred = torch.round(torch.sigmoid(y_logits))

device = "cuda" if torch.cuda.is_available() else "cpu"

Stall Value: No
First Sample Tensor: tensor([ 13.0000,  13.0000,  13.0000,   0.0000,  13.0000,  13.0000,  13.0000,
         13.0000, -76.0000, -76.0000, -81.0000, -76.0000, -78.5000, -76.0000,
        -76.0000,  -7.0000,  -7.0000, -12.0000,  -7.0000,  -9.5000,  -7.0000,
         -7.0000,  12.0000,  12.0000,   7.0000,  12.0000,   9.5000,  12.0000,
         12.0000])


In [21]:
%%timeit -r 1 -n 1
with torch.no_grad():
    y_pred = torch.round(torch.sigmoid(model(first_sample_tensor))).squeeze()

if device == "cuda":
    prediction = y_pred.cpu().numpy() #if it is cuda, then this, otherwise y_pred.numpy()
else:
    prediction = y_pred.numpy()

if prediction == 0:
    result = "No Stall"
elif prediction == 1:
    result = "Stall"
    
print(result)

No Stall
1.45 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
